In [48]:
import requests, re

url, auth = 'http://131.155.21.174:8083', ('', 'ilovecrypto')
sid, token = 20221886, '3535062df506f93f9ac7da6f483e1da29bb75c56'

req = requests.Session()
req.cookies['token'] = token

r = req.get(url+'/', auth=auth)
ciphertext = bytes.fromhex(re.search('target_ciphertext = ([0-9a-f]+)', r.text).groups()[0])

def oracle(cipher):
    assert hasattr(cipher,'hex') and len(cipher) % 16 == 0 and len(cipher) >= 32
    r = req.post(url+'/oracle', auth=auth, data={'cipher': cipher.hex()})
    return bool(['False', 'True'].index(r.text.strip()))

def validate(plaintext):  # use this to submit your plaintext once you've decrypted it
    if type(plaintext) == bytes:
        try: plaintext = plaintext.decode()
        except UnicodeDecodeError: raise Exception('plaintext is printable ASCII; something is amiss.')
    assert type(plaintext) == str
    r = req.post(url+'/validate', auth=auth, data={'plaintext': plaintext})
    print('\x1b[32m{}\x1b[0m'.format(r.text.strip()))

In [49]:
# let's try the oracle with a correct and invalid ciphertext.

res1 = oracle(ciphertext)
print('oracle (correct): {}'.format(res1))

# let's flip a bit
ciphertext_invalid = ciphertext[:-1] + bytes([ciphertext[-1] ^ 1])
res2 = oracle(ciphertext_invalid)
print('oracle (invalid): {}'.format(res2))


oracle (correct): True
oracle (invalid): False


In [87]:
ciphertext_blocks = []
i = 0
while i < len(ciphertext):
    block = ciphertext[i:i+16:]
    ciphertext_blocks.append(block)
    i += 16
ciphertext_blocks

[b'\xff4\x8aJ\xf8\xb4\xb3*\xf2\x94\xef\x94\x88t\x12C',
 b'\xfb\xb8\xbf\xd3\xcc@\x91\x8f\x1d\xe8Z\xbb\x10\xe1W\x15',
 b'3\xd4/\xa7`\t?C\xee\xea\xf1\x85}\xe0\xca\xc4',
 b"\xe8\x80o\x06\x97Sb8\xa8T'\xd7\xdb9?\x90",
 b'\xe0H\xaf\xdb\x16\xf7/b\x8e6\xf7}\xfc\x98\r\x9b']

In [91]:
def block_to_arr(block):
    padding_block_arr = []
    for byte in padding_block:
        padding_block_arr.append(bytes([byte]))
    return padding_block_arr

In [97]:
padding_block = ciphertext_blocks[3]
# print(f"original \n {ciphertext}")
ciphertext_blocks_tmp = ciphertext_blocks
i = 1
res = False
while not res:
    padding_block_tmp = padding_block
    # ciphertext_blocks_tmp = ciphertext_blocks.copy()
    padding_block_tmp_arr = block_to_arr(padding_block_tmp)
    padding_block_tmp_arr[-i] = bytes([padding_block[-i] ^ 1])
    padding_block_tmp = b"".join(padding_block_tmp_arr)
    ciphertext_blocks_tmp[3] = padding_block_tmp
    ciphertext_tmp = b"".join(ciphertext_blocks_tmp)
    # print(f"try \n{ciphertext_tmp}")
    res = oracle(ciphertext_tmp)
    i += 1
print(f"Found {i-2}")
print(oracle(ciphertext_tmp))
ciphertext_tmp

Found 3
True


b"\xff4\x8aJ\xf8\xb4\xb3*\xf2\x94\xef\x94\x88t\x12C\xfb\xb8\xbf\xd3\xcc@\x91\x8f\x1d\xe8Z\xbb\x10\xe1W\x153\xd4/\xa7`\t?C\xee\xea\xf1\x85}\xe0\xca\xc4\xe8\x80o\x06\x97Sb8\xa8T'\xd7\xda9?\x90\xe0H\xaf\xdb\x16\xf7/b\x8e6\xf7}\xfc\x98\r\x9b"